# Importing library:

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import requests
pd.set_option('display.max_columns', None)


# Difining funtion:

In [2]:
def get_Product_name(soup):
    try:
        Product_name = soup.find('span',attrs = {'class':'VU-ZEz'}).text.strip()
    except AttributeError:
        Product_name = ''
    return Product_name
def get_price(soup):
    try:
        price = soup.find('div',attrs = {'class':'Nx9bqj CxhGGd'}).text.strip()
    except AttributeError:
        price = ''
    return price
def get_rating(soup):
    try:
        rating = soup.find('div',attrs = {'class':'XQDdHH'}).text.strip()
    except AttributeError:
        rating = ''
    return rating
def get_Delivery_date(soup):
    try:
        Delivery_date = soup.find('span',attrs = {'class':'Y8v7Fl'}).text.strip()
    except AttributeError:
        Delivery_date = ''
    return Delivery_date
def get_num_of_rating(soup):
    try:
        num_of_rating = soup.find('div',attrs = {'class':'row j-aW8Z'}).text.strip()
    except AttributeError:
        num_of_rating = ''
    return num_of_rating


In [7]:
# Initialize an empty list to store all the product data
all_data = []
HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36','Accept-Language':'en-Us,en;q=0.5'})
# Iterate over first 5  pages
for i in range(2, 7):
    # Construct the URL for the search page
    url = 'https://www.flipkart.com/search?q=phone&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page=' + str(i)
    
    # Send a GET request to the URL with custom headers
    page = requests.get(url, headers=HEADERS)
    
    # Parse the HTML content of the page using BeautifulSoup
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # Find all the links of the product pages on the search page
    links = soup.find_all('a', attrs={'class': 'CGtC98'})
    
    # Iterate over each product link
    for link in links:
        # Send a GET request to the product page
        new_page = requests.get('https://flipkart.com' + link.get('href'), headers=HEADERS)
        
        # Parse the HTML content of the product page using BeautifulSoup
        new_soup = BeautifulSoup(new_page.content, 'html.parser')
        
        # Initialize a dictionary to store product data
        product_data = {
            'Product_name': get_Product_name(new_soup),
            'price': get_price(new_soup),
            'rating': get_rating(new_soup),
            'Delivery_date': get_Delivery_date(new_soup),
            'num_of_rating': get_num_of_rating(new_soup)
        }
        
        # Extract specification data for the product
        table_data = new_soup.find('div', attrs={'class': '_1OjC5I'}).find_all('table', attrs={'class': '_0ZhAN9'})
        for table in table_data:
            column = [td.text.strip() for td in table.find_all('td', attrs={'class': '+fFi1w col col-3-12'})]
            row = [td.text.strip() for td in table.find_all('td', attrs={'class': 'Izz52n col col-9-12'})]
            product_data.update(dict(zip(column, row)))
        
        # Append the product data to the list of all_data
        all_data.append(product_data)

# Create a DataFrame from the collected product data
df = pd.DataFrame(all_data)
# extrating the data frame as CSV file.
df.to_csv('web scraping Flipkart mobile data.csv',header=True, index=False)


In [9]:
df.head()

,Product_name,price,rating,Delivery_date,num_of_rating,In The Box,Model Number,Model Name,Color,Browse Type,SIM Type,Hybrid Sim Slot,Touchscreen,OTG Compatible,Quick Charging,Sound Enhancements,Display Size,Resolution,Resolution Type,GPU,Display Type,Other Display Features,Operating System,Processor Brand,Processor Type,Processor Core,Primary Clock Speed,Secondary Clock Speed,Operating Frequency,Internal Storage,RAM,Expandable Storage,Supported Memory Card Type,Memory Card Slot Type,Primary Camera Available,Primary Camera,Primary Camera Features,Secondary Camera Available,Secondary Camera,Secondary Camera Features,Flash,HD Recording,Full HD Recording,Video Recording,Video Recording Resolution,Digital Zoom,Frame Rate,Dual Camera Lens,Speaker Phone,Network Type,Supported Networks,Internet Connectivity,3G,Micro USB Version,Bluetooth Support,Bluetooth Version,Wi-Fi,Wi-Fi Version,Wi-Fi Hotspot,USB Connectivity,Audio Jack,GPS Support,Smartphone,SIM Size,SMS,Sensors,Upgradable Operating System,Other Features,GPS Type,FM Radio,FM Radio Recording,Battery Capacity,Width,Height,Depth,Weight,Warranty Summary,Domestic Warranty,SAR Value,Call Log Memory,Image Editor,Call Wait/Hold,Conference Call,Hands Free,Call Divert,Phone Book,Call Timer,Call Records,GPRS,Micro USB Port,Mini HDMI Port,NFC,USB Tethering,Infrared,Map Support,Touchscreen Type,User Interface,Instant Message,Removable Battery,MMS,Keypad,Voice Input,Predictive Text Input,Browser,Music Player,Battery Type,Warranty Service Type,Total Memory,EDGE,User Memory,Additional Content,Tertiary Clock Speed,Optical Zoom,Graphics PPI,HD Game Support,Display Colors,Video Call Support,Speed Dialing,Logs,3G Speed,Pre-installed Browser,Mini USB Port,Social Networking Phone,Business Phone,SIM Access,Series,Ringtones Format,DLNA Support,Audio Formats,Video Formats,Dual Battery,Covered in Warranty,Supported Languages,Important Apps,TV Out,Mobile Tracker,Talk Time,Not Covered in Warranty,Phone Book Memory,Java Application,JAVA Support,Keypad Type,SMS Memory,Games
0,"MOTOROLA g24 Power (Glacier Blue, 128 GB) (4 ...","₹7,999",4.2,"10 May, Friday","6,972 Ratings &","Handset, 33W TurboPower Charger, USB Type-C Ca...",PB1C0001IN,g24 Power,Glacier Blue,Smartphones,Dual Sim,No,Yes,Yes,Yes,"Stereo speakers, Dolby Atmos Support",16.76 cm (6.6 inch),1612 x 720 Pixels,HD+,Arm Mali-G52 MP2,HD+ IPS LCD Display,90Hz Refresh Rate,Android 14,Mediatek,Helio G85,Octa Core,2 GHz,1.7 GHz,"4G LTE: B1/B2/B3/B5/B7/B8/B20/B28/B38/B40/B41,...",128 GB,4 GB,1 TB,MicroSD,Dedicated Slot,Yes,50MP + 2MP,Dual Camera Setup: 50MP Main Camera (f/1.8 Ape...,Yes,16MP Front Camera,"Front Camera Setup: 16MP (f/2.4 Aperture, 1.0u...",Single LED Flash,Yes,Yes,Yes,"Rear Camera: Main Camera (FHD (at 30 fps)), Ma...",Upto 4X,30 fps,Primary Camera,Yes,"2G, 3G, 4G","4G LTE, GSM, UMTS","4G, 3G, Wi-Fi",Yes,Type C Port (USB 2.0),Yes,v5.2,Yes,Wi-Fi 802.11 a/b/g/n/ac (2.4 GHz | 5 GHz),Yes,Yes,3.5mm,Yes,Yes,Nano Sim,Yes,"Accelerometer, Proximity Sensor, Ambient Light...","1 OS Upgrade, 3 Years SMRs","4 GB RAM Boost, 30W Device Charging Capable, W...","GPS, A-GPS, GLONASS, GALILEO, LTEPP, SUPL",Yes,Yes,6000 mAh,74.53 mm,163.49 mm,8.99 mm,196 g,1 Year on Handset and 6 Months on Accessories,1 Year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"REDMI 13C (Stardust Black, 128 GB) (4 GB RAM)","₹7,699",4.3,"10 May, Friday","12,122 Ratings &","Handset, 10W Charger, USB Type-C Cable, Sim Ej...",MZB0FOPIN,13C,Stardust Black,Smartphones,Dual Sim,No,Yes,Yes,Yes,NaN,17.12 cm (6.74 inch),1650 x 720 Pixels,HD+,Arm Mali-G52 MC2,HD+ 90Hz Display,Corning Gorilla Glass 3 Protection,Android 13,Mediatek,Helio G85,Octa Core,2 GHz,1.8 GHz,"2G GSM: B2/B3/B5/B8, 3G WCDMA: B1/B5/B8, 4G LT...",128 GB,4 GB,1 TB,NaN,NaN,Yes,50MP Rear Camera,"Triple AI Camera Set